In [36]:
import requests
from bs4 import BeautifulSoup
import datetime
import time
import threading
import queue

In [13]:
%%capture
pip install --upgrade firebase-admin 


In [37]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("jsonFile/service-account.json")
    firebase_admin.initialize_app(cred)
    
db = firestore.client()

In [38]:
docs = db.collection(u'cguscholar').document('faE3_ksAAAAJ').collection(u'updateTime').order_by(u'h_index.All', direction=firestore.Query.DESCENDING).limit(3).get()
for doc in docs:
    print(f'{doc.id} => {doc.to_dict()}')

2021-09-23 15:51:15 => {'h_index': {'Since2016': '55', 'All': '74'}, 'citations': {'Since2016': '54991', 'All': '118747'}, 'i10_index': {'Since2016': '127', 'All': '156'}}
2021-09-22 17:24:22 => {'h_index': {'Since2016': '55', 'All': '74'}, 'a': 'a', 'citations': {'Since2016': '54991', 'All': '118747'}, 'i10_index': {'Since2016': '127', 'All': '156'}}
2021-09-22 16:44:58 => {'h_index': {'Since2016': '55', 'All': '74'}, 'a': 'z', 'citations': {'Since2016': '54991', 'All': '118747'}, 'i10_index': {'Since2016': '127', 'All': '156'}}


In [39]:
doc_ref = db.collection(u'cguscholar').order_by('name')

doc = doc_ref.get()

print(doc)


[<google.cloud.firestore_v1.base_document.DocumentSnapshot object at 0x0000021AEF8CDC70>, <google.cloud.firestore_v1.base_document.DocumentSnapshot object at 0x0000021AEF8CD7F0>, <google.cloud.firestore_v1.base_document.DocumentSnapshot object at 0x0000021AEF8CD610>]


In [52]:
docs = db.collection(u'cguscholar').order_by(u'updateTime').get()

for doc in docs:
    print(f'{doc.id}')

faE3_ksAAAAJ
g-_ZXGsAAAAJ
bjEpXBoAAAAJ


In [41]:
def currentTime():
    now = datetime.datetime.now()
    currentTime = now.strftime("%Y-%m-%d %H:%M:%S")
    return currentTime

In [42]:
#personal detail
def personalData(soup):
    info = {}
    content = ''.join(soup.find('meta', property = 'og:image')['content']) #list to string
    id = content.split('user=')[1].split('&citpid')[0] 

    d = soup.find('div', id = 'gsc_prf_i')

    #name
    info['name'] = d.find('div', id = 'gsc_prf_in').text

    #university
    info['university'] = d.find('a', class_ = 'gsc_prf_ila').text

    #email
    email = d.find('div', id = 'gsc_prf_ivh').text.split(' ')[1]
    info['email'] = email

    #picture
    info['picture'] = soup.find('div', id = 'gsc_prf_pua').find('img')['src'] 
    
    label = []
    for p in soup.find_all('a', class_='gsc_prf_inta gs_ibl'):

        label.append(p.text)
    info['label'] = label
    info['updateTime'] = currentTime()
    return info


In [43]:
def citeBy(soup):
    citeBy = {}
    citations = {}
    h_index = {}
    i10_index = {}
    def cited(status, value):

        if status / 2 < 1:        
            if status % 2 == 0:
                citations['All'] = value
            else:
                citations['Since2016'] = value
            citeBy['citations'] = citations

        if status / 2 < 2:
            if status % 2 == 0:
                h_index['All'] = value
            else:
                h_index['Since2016'] = value
            citeBy['h_index'] = h_index

        if status / 2 < 3:
            if status % 2 == 0:
                i10_index['All'] = value
            else:
                i10_index['Since2016'] = value
            citeBy['i10_index'] = i10_index
            
    count_d = 0
    for d in soup.find_all('td', class_='gsc_rsb_std'):

        cited(count_d, d.text)
        count_d = count_d + 1       
    
    return citeBy

In [44]:
def result(soup, ID):
    infos = {}
    infos['id'] = ID
    infos['personalData'] = personalData(soup)
    infos['cited'] = citeBy(soup)

    return infos

In [45]:
def personalPage(id):
    url = 'https://scholar.google.com.tw/citations?hl=zh-TW&user=' + id
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    return result(soup, id)

In [46]:
import json
def jsontransfer(items):
    data = json.dumps(items)
    jsonStr = json.loads(data) 
    return jsonStr

In [47]:
def updatePersonal(userID):
    items = jsontransfer(personalPage(userID))
    print(items)
    ref = db.collection(u'cguscholar').document((items['id']))
    ref.collection(u'updateTime').document((items['personalData']['updateTime'])).set(items['cited'])
    ref.set(items['personalData'])

In [53]:
# import time
# import threading
# import queue
def userDataList(limit):
    userDataList = []
    # 建立佇列
    ID_queue = queue.Queue()

    # 將資料放入佇列
    users_ref = db.collection(u'cguscholar').order_by(u'updateTime').limit(limit)
    docs = users_ref.get()

    for doc in docs:
        #print(u'{}'.format(doc.id))
        ID_queue.put((doc.id))

    # Worker 類別，負責處理資料
    class Worker(threading.Thread):
      def __init__(self, queue, num):
        threading.Thread.__init__(self)
        self.queue = queue
        self.num = num

      def run(self):
        while self.queue.qsize() > 0:
          # 取得新的資料
          userID = self.queue.get()

          # 處理資料
          #print("Worker %d: %s" % (self.num, userID))
          updatePersonal(userID)

          time.sleep(1)

    # 建立兩個 Worker
    my_worker1 = Worker(ID_queue, 1)
    my_worker2 = Worker(ID_queue, 2)

    # 讓 Worker 開始處理資料
    my_worker1.start()
    my_worker2.start()

    # 等待所有 Worker 結束
    my_worker1.join()
    my_worker2.join()

    print("Done.")
    


In [54]:
userDataList(3)

{'id': 'faE3_ksAAAAJ', 'personalData': {'name': 'Jeffrey M. Wooldridge', 'university': 'Michigan State University', 'email': 'msu.edu', 'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=faE3_ksAAAAJ&citpid=2', 'label': ['econometrics', 'panel data', 'treatment effects', 'clustering', 'economics'], 'updateTime': '2021-09-23 15:58:30'}, 'cited': {'citations': {'All': '118747', 'Since2016': '54991'}, 'h_index': {'All': '74', 'Since2016': '55'}, 'i10_index': {'All': '156', 'Since2016': '127'}}}
{'id': 'bjEpXBoAAAAJ', 'personalData': {'name': 'Matti Pietikäinen', 'university': 'University of Oulu', 'email': 'oulu.fi', 'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=bjEpXBoAAAAJ&citpid=2', 'label': ['computer vision', 'pattern recognition', 'biometrics', 'affective computing', 'artificial intelligence'], 'updateTime': '2021-09-23 15:58:30'}, 'cited': {'citations': {'All': '73264', 'Since2016': '37921'}, 'h_index': {'All': 

In [88]:
# jsonFile = open("jsonFile/personalData.json", "w")
# jsonFile.write(jsonStr)
# jsonFile.close()